In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import keras
import keras_tuner
from dataclasses import dataclass
from typing import List
from keras import layers, ops
from keras.utils import timeseries_dataset_from_array
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
%matplotlib inline

from processing import split_indices, make_stationary, inverse_stationary, scale, unscale, sequence, Stationarizer, Normalizer
from tuner import LSTMLayerHyperparams, ModelHyperparams
from plotting import visualize_loss, show_plot
from model_builder import get_model, build_model, get_tuner

In [7]:
data = pd.read_csv("data/YF_AAPL.csv")["Adj Close"].to_numpy().reshape(-1, 1)

train_idx = int(0.7 * np.shape(data)[0])
val_idx = int(0.85 * np.shape(data)[0])

train_data = data[:train_idx]
val_data = data[train_idx:val_idx]
test_data = data[val_idx:]

In [8]:
#train_stater = Stationarizer()
#val_stater = Stationarizer()
#normer = Normalizer()
normer = MinMaxScaler(feature_range=(0, 1), clip=True)
#train_data = train_stater.fit_transform(train_data)
#val_data = val_stater.fit_transform(val_data)
train_data = normer.fit_transform(train_data)
val_data = normer.transform(val_data)

In [9]:
train_ds = timeseries_dataset_from_array(
    train_data[:-20],
    train_data[20:],
    sequence_length=20,
    batch_size=8,
    shuffle=True
)
val_ds = timeseries_dataset_from_array(
    val_data[:-20],
    val_data[20:],
    sequence_length=20,
    batch_size=8,
    shuffle=False
)

for batch in train_ds.take(1):
    inputs, targets = batch

print("Input shape:", inputs.numpy().shape)
print("Target shape:", targets.numpy().shape)

Input shape: (8, 20, 1)
Target shape: (8, 1)


2024-08-02 16:03:24.727248: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [ ]:
tuner = get_tuner()
tuner.search(train_ds, epochs=20, validation_data=val_ds)

Trial 6 Complete [00h 00m 51s]
mse: 0.00013938720803707838

Best mse So Far: 0.00013938720803707838
Total elapsed time: 00h 07m 52s

Search: Running Trial #7

Value             |Best Value So Far |Hyperparameter
2                 |1                 |num_layers
32                |32                |units_0
relu              |tanh              |activation_0
sigmoid           |relu              |recurrent_activation_0
0.5               |0                 |dropout_0
0                 |0.1               |recurrent_dropout_0
relu              |sigmoid           |activation_1
0.0010989         |0.037954          |lr
16                |128               |units_1
relu              |relu              |recurrent_activation_1
0                 |0.2               |dropout_1
0.1               |0.4               |recurrent_dropout_1
relu              |sigmoid           |activation_2

Epoch 1/20
535/535 ━━━━━━━━━━━━━━━━━━━━ 6s 8ms/step - loss: 0.1484 - mse: 0.1484 - val_loss: 3.1713 - val_mse: 3.1713
